In [ ]:
import glob

def ensemble_output():
        # store file name and their emsemble file name 
        output1 = {}
        # iterate file from the folder storing page and their text output
        for image in glob.glob(f"{images_output}/*.txt"):
            print(image)
            # get new file name using method
            newfile = get_filename(image)
            # open image in read mode
            with open(image) as fp:
                # read the file
                text = fp.read()
                # open new file and append text of files for the same name 
                with open(f'{ensemble_text_output}/{newfile}.txt', 'a') as file:
                    # to separate the text of multiple configuration using ---
                    file.write(text+"\n---"+"\n")

In [10]:
import glob
import os

def ensemble_output():

    for file in glob.glob('hotel_stay_preprocessed4\\*.txt'):
        filename = os.path.split(file)[-1].split('.txt')[0]
        ind = filename.index('page')
        filename = filename[:ind+5]
        print(filename)
        with open(file, encoding="utf-8") as fp:
                # read the file
                text = fp.read()
                # open new file and append text of files for the same name 
                with open('ensemble_files_hotel_stay4\\{}.txt'.format(filename), 'a') as file:
                    # to separate the text of multiple configuration using ---
                    file.write(text+"\n---"+"\n")


In [11]:
ensemble_output()

CCF_000141_page0
CCF_000141_page0
CCF_000141_page1
CCF_000141_page1
CCF_000141_page2
CCF_000141_page2
CCF_000141_page3
CCF_000141_page3
CCF_000141_page4
CCF_000141_page4
CCF_000141_page5
CCF_000141_page5
CCF_000141_page6
CCF_000141_page6
Halaplaza_Bill_page0
Halaplaza_Bill_page0
invoice_1_page0
invoice_1_page0
invoice_1_page1
invoice_1_page1
invoice_1_page2
invoice_1_page2
Invoice_Vitheya___Prabhakaran_page0
Invoice_Vitheya___Prabhakaran_page0
Manikandan_Anbalagan_12Jan-14th_Jan_Stay_Amsterdam_page0
Manikandan_Anbalagan_12Jan-14th_Jan_Stay_Amsterdam_page0
Manikandan_Anbalagan_12Jan-14th_Jan_Stay_Amsterdam_page1
Manikandan_Anbalagan_12Jan-14th_Jan_Stay_Amsterdam_page1
Netlink_Digital_Bill_No._119770,119771,119773,119774,119799,119805_page2
Netlink_Digital_Bill_No._119770,119771,119773,119774,119799,119805_page2
Netlink_Digital_Bill_No._119770,119771,119773,119774,119799,119805_page2
Netlink_Digital_Bill_No._119770,119771,119773,119774,119799,119805_page2
Netlink_Digital_Bill_No._119770,

In [21]:
print(len(glob.glob('ensemble_files\\*.txt')))

144


In [2]:
import dateparser
from datetime import datetime
import glob
from prettyprinter import pprint
import pandas as pd
import cv2
import os
import re
import airportsdata

def is_date(string):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    # formats = ['%d %b']
    # try: 
    #     print(dateparser.parse(string, date_formats = formats))
    #     return True

    # except ValueError:
    #     return False

    formats = ["%d %b %Y", "%d%b %Y", "%d%b%Y"]
 
    # checking if format matches the date
    res = False
    
    # using try-except to check for truth value
    for format in formats:
        try:
            res = bool(datetime.strptime(string, format))
            return res
        except ValueError:
            res = False
    return res

# def contains_date(string):
#     date_formats = ['%d%b %Y', '%d %b %Y']
#     for format in date_formats:
#         try:
#             datetime.strptime(re.search(format, string).group(), format)
#             return True
#         except (AttributeError, ValueError):
#             pass
#             # print(e)
#     return False


def darken_text(image, k, filter):
    try:
        img = cv2.imread(image)
        if filter == 'gray':
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if filter == 'rgb':
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # k = 3
        result = 255*(img/255)**k
        cv2.imwrite('result.jpg', result)
        return 'result.jpg'
    except:
        return image

def extract_name(text):
    name = ''
    flag = 0
    for i in range(len(text)-1):
        if 'name' in text[i].lower() and 'sector' not in text[i].lower():
            flag = 1
            if ':' in text[i]:
                name =  text[i].strip().split('Name:')[-1].strip()
                break
            if '-' in text[i]:
                name = text[i].strip().split('Name - ')[-1].strip()
                break
    if flag == 0:
        for i in range(len(text)):
            if 'pax name' in text[i].lower():
                if 'mr' in text[i-1].lower():
                    name = text[i-1].strip()
                    break

    if 'adt' in name.lower():
        ind = name.lower().index('adt')
        name = name[:ind+3]
        if ')' not in name:
            if '.' in name[-1]:
                name = name.replace('.', ')')
            else:
                name += ')'
        else:
            i = name.index(')')
            name = name[:i+1]
    
    if name[-2:].lower() == 'ad':
        name += 'T)'

    extras = ['Name - ', 'Names', 'name.', 'Name -']

    for extra in extras:
        if extra in name:
            name = name.strip(extra)
    
    if '|' in name:
        if name.index('|') == 0:
            name = name.replace('|', '')
        else:
            name = name.replace('|', 'I')

    return name

def remove_exc(sc):
    if len(sc) == 3 and '!' in sc:
        sc = sc.replace('!', 'I')
    elif len(sc) > 3 and '!' in sc:
        sc = sc.replace('!', '')
    if '(' in sc:
        i = sc.index('(')
        sc = sc[:i]
    if '.' in sc:
        sc = sc.replace('.', '')
    if '=' in sc:
        sc = sc.replace('=', '')
    return sc
    

def extract_source_city(text):
    # print('source city-------------')
    # pprint(text)
    source_cities = []
    destination_cities = []
    flag1 = 0
    debarred = ['spicejet', 'india', 'sg', 'al', 'airline', 'airlines', 'i5', 'asia', '15', 'jet', 'u2', 'airway', '6e', '6684)', '‘qr']
    for i in range(len(text)-1):
        if 'sector' in text[i].lower() or 'pax' in text[i].lower() or 'travel' in text[i].lower():
            flag1 = 1
            sc = ''
            dc = ''
            flag2 = 0
            for j in range(i+1, len(text)):
                # print(text[j])
                words = text[j].split()
                # print(words)
                for word in words:
                    # print(word)
                    try:
                        if '-' in word:
                            ind = word.find(next(filter(str.isalpha, word)))
                            word = word[ind:]
                            # print(word)
                    except:
                        pass
                        
                    if 'airlines' in word.split('-')[0].lower():
                        # print('AIRLINESSSS')
                        continue
                    if 'indigo' in word.split('-')[0].lower() or 'charge' in word.split('-')[0].lower():
                        # print('INDIGOOOOO')
                        break
                    #  and not any(chr.isdigit() for chr in word)
                    if '-' in word and len(word.split('-')[0]) >= 3 and '(' not in word.split('-')[0] and '{' not in word:
                        
                        if '(' in word:
                            ind = word.index('(')
                            word = word[:ind]
                        ch_ind = word.find(next(filter(str.isalpha, word)))
                        # print(ch_ind)
                        word = word[ch_ind:]
                        
                        sc = word.split('-')[0]
                        sc = remove_exc(sc)
                        if sc.lower() not in debarred and len(sc) == 3:
                            source_cities.append(sc.upper())

                        dc = word.split('-')[1]
                        dc = remove_exc(dc)
                        if dc.lower() not in debarred and len(dc) == 3:
                            destination_cities.append(dc.upper())
                        continue

        if flag1 == 1:
            break
    
    return source_cities, destination_cities

def extract_start_date(text):
    start_dates = []
    flag1 = 0
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    for i in range(len(text)-1):
        if 'travel date' in text[i].lower():
            flag1 = 1
            for j in range(i+1, len(text)):
                words = text[j].split('|')
                for word in words:
                    # print(word+'---->'+str(is_date(word.strip())))
                    if is_date(word.strip(" ,")):
                        date = word.strip(" ,")
                        date = ''.join(date.split())
                        date = date[:2]+" "+date[2:5]+" "+date[5:]
                        # print('DATE-------->', date)
                        # if len(word[i]) == 5:
                        #     date = word[i][:2]+" "+word[i][2:]+" "+word[i+1]
                        # elif len(word[i]) == 7:
                        #     date = word[i-1]+" "+word[i][:3]+" "+word[i][3:]
                        # elif len(word[i]) == 9:
                        #     date = word[i][:2]+" "+word[i][2:5]+" "+word[i][5:]
                        # elif len(word[i]) == 3:
                        #     date = word[i-1]+" "+word[i]+" "+word[i+1]
                        # if date[:2].isnumeric():
                        #     if '.' in date:
                        #         date = date.replace('.', '')
                        #     start_dates.append(date)
                        date = datetime.strptime(date, '%d %b %Y').strftime('%Y-%m-%d')
                        start_dates.append(date)
                        continue
                    else:
                        flag2 = 0
                        date = ''
                        debarred = ['remarks:', 'separate', 'connaught']
                        word = word.split()
                        for i in range(len(word)):
                            for month in months:
                                if month in word[i].lower() and word[i].lower() not in debarred:
                                    flag2 = 1
                                    ind = word[i].lower().index(month)
                                    # print(word[i])
                                    word[i] = word[i].strip('.')
                                    try:
                                        if not word[i][ind-1].isalpha() or not word[i-1].isalpha():
                                            if len(word[i]) == 5:
                                                date = word[i][:2]+" "+word[i][2:]+" "+word[i+1]
                                            elif len(word[i]) == 7:
                                                date = word[i-1]+" "+word[i][:3]+" "+word[i][3:]
                                            elif len(word[i]) == 9:
                                                date = word[i][:2]+" "+word[i][2:5]+" "+word[i][5:]
                                            elif len(word[i]) == 3:
                                                date = word[i-1]+" "+word[i]+" "+word[i+1]
                                            if date[:2].isnumeric():
                                                if '.' in date:
                                                    date = date.replace('.', '')
                                                date = datetime.strptime(date, '%d %b %Y').strftime('%Y-%m-%d')
                                                start_dates.append(date)
                                                break
                                    except:
                                        pass
                            if flag2 == 1:
                                continue
        if flag1 == 1:
            break

    return start_dates


            
ticket_details = {}
names = []
source_cities = []
destination_cities = []
start_dates = []
end_dates = []

final = []
for foutput in glob.glob('final_output_ensemble\\*'):
    final.append(os.path.split(foutput)[-1])

for file in glob.glob('C:\\air_ticket\\ensemble_files\\*.txt'):
    temp_filename = os.path.split(file)[-1]
    if temp_filename not in final:
        name_flag = 0
        s_city_flag = 0
        d_city_flag = 0
        start_date_flag = 0
        end_date_flag = 0
        with open(file) as fp:
            texts = fp.read().split("---\n")
            # pprint(texts)
            for text in texts:
                text = text.splitlines()
                text = [line for line in text if not line.isspace() and len(line) > 0]
                # pprint(text)
                # print('--------------------------------------------')
                filename = os.path.split(file)[-1].split('.txt')[0]

                # pprint(text)
                print(filename)
                # print()

                # name = extract_name(text)
                # print(name)
                # names = []
                # source_cities = []
                # destination_cities = []
                # start_dates = []
                # end_dates = []

                name = extract_name(text)
                if len(name) == 0:
                    ## call darken text function and call extract name function again
                    img_file = 'C:\\air_ticket\\images2\\{}.jpg'.format(filename)
                    dark_image = darken_text(img_file, 3, 'gray')
                    command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
                    os.system(command)
                    with open('text_output.txt', 'r', encoding='utf-8') as fp:
                        text_for_name = fp.read()
                    text_for_name = text_for_name.splitlines()
                    text_for_name = [line for line in text_for_name if not line.isspace() and len(line) > 0]
                    # print('NAME---------------------')
                    # pprint(text)
                    name = extract_name(text_for_name)

                if len(name) == 0:
                    img_file = 'C:\\air_ticket\\images2\\{}.jpg'.format(filename)
                    dark_image = darken_text(img_file, 3, 'gray')
                    command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
                    os.system(command)
                    with open('text_output.txt', 'r', encoding='utf-8') as fp:
                        text_for_name2 = fp.read()
                    text_for_name2 = text_for_name2.splitlines()
                    text_for_name2 = [line for line in text_for_name2 if not line.isspace() and len(line) > 0]
                    # print('NAME---------------------')
                    # pprint(text)
                    name = extract_name(text_for_name2)

                if len(name) == 0:
                    img_file = 'C:\\air_ticket\\images2\\{}.jpg'.format(filename)
                    dark_image = darken_text(img_file, 4, 'gray')
                    command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
                    os.system(command)
                    with open('text_output.txt', 'r', encoding='utf-8') as fp:
                        text_for_name3 = fp.read()
                    text_for_name3 = text_for_name3.splitlines()
                    text_for_name3 = [line for line in text_for_name3 if not line.isspace() and len(line) > 0]
                    # print('NAME---------------------')
                    # pprint(text)
                    name = extract_name(text_for_name3)
                
                if len(name) == 0:
                    img_file = 'C:\\air_ticket\\images2\\{}.jpg'.format(filename)
                    dark_image = darken_text(img_file, 5, 'rgb')
                    command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
                    os.system(command)
                    with open('text_output.txt', 'r', encoding='utf-8') as fp:
                        text_for_name4 = fp.read()
                    text_for_name4 = text_for_name4.splitlines()
                    text_for_name4 = [line for line in text_for_name4 if not line.isspace() and len(line) > 0]
                    # print('NAME---------------------')
                    # pprint(text)
                    name = extract_name(text_for_name4)

                if len(name) == 0:
                    img_file = 'C:\\air_ticket\\images2\\{}.jpg'.format(filename)
                    dark_image = darken_text(img_file, 6, 'gray')
                    command = 'tesseract {} text_output -l eng --psm 4'.format(dark_image)
                    os.system(command)
                    with open('text_output.txt', 'r', encoding='utf-8') as fp:
                        text_for_name5 = fp.read()
                    text_for_name5 = text_for_name5.splitlines()
                    text_for_name5 = [line for line in text_for_name5 if not line.isspace() and len(line) > 0]
                    # print('NAME---------------------')
                    # pprint(text)
                    name = extract_name(text_for_name5)

                # print('Name ----> ', name)
                source_city, destination_city = extract_source_city(text)
                # print('Source City ----> ', source_city)
                # print('Destination City ----> ', destination_city)
                start_date = extract_start_date(text)
                end_date = extract_start_date(text)
                # print('Start Date ----> ', start_date)
                # print('End Date ----> ', end_date)
                # if name is not None:
                    # names.append(name*len(start_dates))
                # print(names)
                
                text_file = 'interm_output_ensemble\\{}.txt'.format(filename)
                with open(text_file, 'a', encoding='utf-8') as fp:
                    if not name.strip():
                        name = 'None'
                    fp.write(name+"\n")
                    fp.write(str(source_city)+"\n") 
                    fp.write(str(destination_city)+"\n") 
                    fp.write(str(start_date)+"\n")
                    fp.write(str(end_date)+"\n")
                    fp.write(str(filename)+"\n") 
                    fp.write('----------------------------------------------\n')

                ind = filename.index('page')
                filename = filename[:ind+5]
                final_output_file = 'final_output_ensemble\\{}.txt'.format(filename)
                with open(final_output_file, 'a') as fp:
                    if name_flag == 0 and name != '':
                        name_flag = 1
                        fp.write(name+"\n")
                    if s_city_flag == 0 and len(source_city) > 0:
                        s_city_flag = 1
                        fp.write(str(source_city)+"\n")
                    if d_city_flag == 0 and len(destination_city) > 0:
                        d_city_flag = 1
                        fp.write(str(destination_city)+"\n")
                    if start_date_flag == 0 and len(start_date) > 0:
                        start_date_flag = 1
                        fp.write(str(start_date)+"\n")
                    if end_date_flag == 0 and len(end_date) > 0:
                        end_date_flag = 1
                        fp.write(str(end_date)+"\n")

            
            # print('---------------------------------------------------------')
        # print(filename)


            # l = len(destination_city)

            # for i in range(len(source_city)):
            #     names.append(name)
            
            # for i in range(len(source_city)):
            #     source_cities.append(source_city[i])
            
            # for i in range(len(destination_city)):
            #     destination_cities.append(destination_city[i])
            
            # for i in range(len(start_date)):
            #     start_dates.append(start_date[i])
                
            # for i in range(len(end_date)):
            #     end_dates.append(end_date[i])

            # # ticket_details['Page'] = filename
            # ticket_details['Name'] = names
            # ticket_details['Source City'] = source_cities
            # ticket_details['Destination City'] = destination_cities
            # ticket_details['Start Date'] = start_dates
            # ticket_details['End Date'] = end_dates

            

        # ticket_details['Name'] = names
        # ticket_details['Start Date'] = start_dates
        # ticket_details['End Date'] = end_dates
        # ticket_details['Source City'] = source_cities
        # ticket_details['Destination City'] = destination_cities

            # if len(start_date) != len(source_city):
            #     print(filename)
            #     print(source_city)
            #     print(start_date)

        # pprint(ticket_details)
        # print(len(ticket_details['Name']))
        # print(len(ticket_details['Source City']))
        # print(len(ticket_details['Destination City']))
        # pprint(ticket_details['Start Date'])
        # print(len(ticket_details['End Date']))

        # keys = ['Name', 'Source City', 'Destination City']
        # # print(len(keys))

        # df = pd.DataFrame(ticket_details)
        # # df.index = keys
        # df.to_csv('ticket_details.csv')
        # print(df)

Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670118_18.05.2022_page8
Yatra_AAAIN233670119_18.05.2022_page0
Yatra_AAAIN233670119_18.05.2022_page0
Yatra_AAAIN233670119_18.05.2022_page0
Yatra_AAAIN233670119_18.05.2022_page0
Yatra_AAAIN233670119_18.05.2022_page0
Yatra_AAAIN233670119_18.05.2022_page0
Yatra_AAAIN233670119_18.05.2022_page0
Yatra_AAAIN2

In [12]:
import glob
from prettyprinter import pprint

final = glob.glob('final_output_ensemble\\*')
final = [os.path.split(filename)[-1] for filename in final]
# pprint(final)
count = 0
for file in glob.glob('ensemble_files\\*'):
    filename = os.path.split(file)[-1]
    # print(filename)
    if filename in final:
        count += 1

print(count)

27


In [13]:
count = 0

for file in glob.glob('final_output_ensemble\\*'):
    count += 1

print(count)

27
